# Transformers. Sentiment. Дообучение

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
from tqdm import tqdm
from collections import Counter

import pandas as pd

In [2]:
max_words = 2000
max_len = 30
num_classes = 1

# Training
epochs = 10
batch_size = 512
print_batch_n = 100

In [3]:
# !pip install transformers

## Сначала сделаем прогноз на val на готовой модели-трансформере SamLowe/roberta-base-go_emotions

In [4]:
from transformers import pipeline

pipe = pipeline("text-classification", model="SamLowe/roberta-base-go_emotions")

config.json:   0%|          | 0.00/1.92k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/380 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [5]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
roberta = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")

In [6]:
pipe("it`s very bad!!!")

[{'label': 'disgust', 'score': 0.2920587658882141}]

In [8]:
df = pd.read_csv('/content/sample_data/data_sentiment/train.csv')

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
df_train, df_val = train_test_split(df, test_size=0.3, random_state = 21)

In [11]:
df_train['label'].value_counts()

0    20837
1     1536
Name: label, dtype: int64

In [12]:
def balance_df_by_target(df, target_name):

    target_counts = df[target_name].value_counts()

    major_class_name = target_counts.argmax()
    minor_class_name = target_counts.argmin()

    disbalance_coeff = int(target_counts[major_class_name] / target_counts[minor_class_name]) - 1

    for i in range(disbalance_coeff):
        sample = df[df[target_name] == minor_class_name].sample(target_counts[minor_class_name])
        df = df.append(sample, ignore_index=True)

    return df.sample(frac=1)

In [13]:
df_for_balancing = df_train
df_train = balance_df_by_target(df_for_balancing, 'label')

df_train['label'].value_counts()

<ipython-input-12-d3c0ac4ac141>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(sample, ignore_index=True)


0    20837
1    19968
Name: label, dtype: int64

In [14]:
df_for_balancing = df_val
df_val = balance_df_by_target(df_for_balancing, 'label')

df_val['label'].value_counts()

<ipython-input-12-d3c0ac4ac141>:12: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(sample, ignore_index=True)


0    8883
1    8472
Name: label, dtype: int64

In [20]:
# ! pip install stop_words

In [21]:
# ! pip install pymorphy2

In [22]:
from string import punctuation
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
import re

In [23]:
sw = set(get_stop_words("en"))
sw_left = set(['no','not', 'nor', "mustn't", "shouldn't","shan't", 'against', "aren't", "can't",
 'cannot',"wouldn't",
 "couldn't","didn't", "doesn't", "don't", "hadn't", "hasn't","haven't", "isn't", "wasn't", "weren't", "won't"
              ])
sw_in = set(['im', 'u', 'will', 'just'])

In [24]:
sw.symmetric_difference_update(sw_left)

In [25]:
sw.update(sw_in)

In [26]:
puncts = set(punctuation)

In [27]:
morpher = MorphAnalyzer()


def preprocess_text(txt):
    txt = str(txt)
    txt = "".join(c for c in txt if c not in puncts)
    txt = txt.lower()
#     txt = re.sub("no\s", "not", 'nore', txt)
    txt = [morpher.parse(word)[0].normal_form for word in txt.split() if word not in sw]
    for word in txt:
        if '@' in word:
            txt.remove(word)
    return " ".join(txt)

In [28]:
df_train['tweet'].iloc[0:1].values

array(["@user this's insane acts by #koreans. they will trample national flag of other nation. it's clearly . #rapeculture "],
      dtype=object)

In [29]:
df_train['tweet'].iloc[0:1].apply(preprocess_text).values

array(['user thiss insane acts koreans trample national flag nation clearly rapeculture'],
      dtype=object)

In [30]:
from tqdm import tqdm
tqdm.pandas()

df_train['tweet'] = df_train['tweet'].progress_apply(preprocess_text)
df_val['tweet'] = df_val['tweet'].progress_apply(preprocess_text)

100%|██████████| 17355/17355 [00:03<00:00, 5636.36it/s]


In [31]:
df_val['tweet'].iloc[0:1].values

array(['user french try stone amp kill indians quebec criminal user user user'],
      dtype=object)

In [32]:
df_val['label'].iloc[0:1].values

array([1])

In [33]:
pipe(str(df_val['tweet'].iloc[0:1].values))[0]['label']

'neutral'

In [ ]:
#предсказания загруженной моделью на валидационном датасете
df_val['pred_pretrtained_model'] = df_val['tweet'].apply(lambda text: pipe(str(text))[0]['label'])

In [ ]:
df_val[df_val['pred_pretrtained_model'] == 'joy'].head()

,id,label,tweet,pred_pretrtained_model
8074,28461,0,ð goal feel good balanced happyâ¤ï¸ð b...,joy
9808,10232,1,user happy new year says fascist pennyloafer n...,joy
6166,19904,0,user happy friday ladies ready weekend shopthe...,joy
7420,13855,0,new trending gif user stephen colbe happy dance,joy
5717,30883,0,get get get enjoy music today free apps free m...,joy


In [ ]:
df_val.to_csv('val_preds_pretrained')

In [ ]:
# df_val = pd.read_csv('val_preds_pretrained')

In [ ]:
df_val['pred_pretrtained_model'].unique()

array(['neutral', 'admiration', 'love', 'fear', 'annoyance', 'amusement',
       'approval', 'gratitude', 'excitement', 'caring', 'disappointment',
       'anger', 'disapproval', 'sadness', 'joy', 'optimism', 'disgust',
       'desire', 'confusion', 'surprise', 'embarrassment', 'remorse',
       'nervousness', 'realization', 'pride', 'curiosity'], dtype=object)

In [ ]:
 vocab_label = {'neutral': 0, 'love': 0, 'disappointment': 1, 'gratitude': 0, 'joy': 0,
       'admiration': 0, 'anger':1, 'excitement': 0, 'annoyance': 1, 'amusement': 0,
       'approval': 0, 'sadness': 1, 'optimism': 0, 'surprise': 0, 'embarrassment': 1,
       'fear': 1, 'remorse': 1, 'confusion': 1, 'caring': 0, 'desire': 0, 'disapproval': 1,
       'curiosity': 0, 'disgust': 1, 'nervousness': 1, 'realization': 0, 'pride': 0}

In [ ]:
df_val['pred_pretrtained_model_label']= df_val['pred_pretrtained_model'].apply(lambda text: vocab_label[text])

In [ ]:
df_val

,id,label,tweet,pred_pretrtained_model,pred_pretrtained_model_label
8070,6209,0,prettyboy mca mulan,neutral,0
2272,14058,0,3 upcoming releases summer remixes user user w...,neutral,0
12501,29054,1,fair amp lovely effect,admiration,0
1850,9632,0,4 days single release late nights band favorit...,neutral,0
3345,25241,0,tgif ff gamedev indiedev indiegamedev squad us...,neutral,0
...,...,...,...,...,...
14925,28466,1,user misogony spectrum voted user,neutral,0
16628,30403,1,user user ð¹user please stop go away amp tak...,neutral,0
14661,22812,1,josephjett fatherhood nor shall engage aberran...,neutral,0
12206,112,1,user not interested linguistics doesnt address...,neutral,0


In [ ]:
from sklearn.metrics import accuracy_score,  f1_score

In [ ]:
acc = accuracy_score(df_val['label'], df_val['pred_pretrtained_model_label'])

In [ ]:
f1 = f1_score(df_val['label'], df_val['pred_pretrtained_model_label'])

In [ ]:
print(f' accuracy on pretrained model balanced classes: {acc}, f1: {f1}')

 accuracy on pretrained model balanced classes: 0.5186401613367906, f1: 0.15735323784547103


Вывод: на готовой обученной модели после проведения балансировки классов accuracy 0,51. При этом показатель f1 низкий.

## Теперь дообучим модель на train датасете и сравним результаты

In [34]:
class TwitterDataset(torch.utils.data.Dataset):

    def __init__(self, txts, labels):
        self._labels = labels

        self.tokenizer = AutoTokenizer.from_pretrained("SamLowe/roberta-base-go_emotions")
        self._txts = [self.tokenizer(text, padding='max_length', max_length=10,
                                     truncation=True, return_tensors="pt")
                      for text in txts]

    def __len__(self):
        return len(self._txts)

    def __getitem__(self, index):
        return self._txts[index], self._labels[index]

In [35]:
y_train = df_train['label'].values
y_val = df_val['label'].values

train_dataset = TwitterDataset(df_train['tweet'], y_train)
valid_dataset = TwitterDataset(df_val['tweet'], y_val)

train_loader = torch.utils.data.DataLoader(train_dataset,
                          batch_size=64,
                          shuffle=True,
                          num_workers=2)
valid_loader = torch.utils.data.DataLoader(valid_dataset,
                          batch_size=64,
                          shuffle=False,
                          num_workers=1)

In [42]:
class RobertaClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super().__init__()
        self.roberta = AutoModelForSequenceClassification.from_pretrained("SamLowe/roberta-base-go_emotions")
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(28, 2)
        self.sigm = nn.Sigmoid()

    def forward(self, x, mask):

        pooled_output = self.roberta(input_ids=x, attention_mask = mask, return_dict=None)
        dropout_output = self.dropout(pooled_output.logits)
        linear_output = self.linear(dropout_output)
        final_layer = self.sigm(linear_output)
        return final_layer

In [43]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [44]:
model = RobertaClassifier().to(device)
criterion = nn.CrossEntropyLoss()

# optimizer = Adam(model.parameters(), lr=0.001)  # полное обучение
optimizer = Adam(model.linear.parameters(), lr=0.001)  # неполное обучение

In [45]:
print(roberta)

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [46]:
print(model)
print("Parameters full train:", sum([param.nelement() for param in model.parameters()]))
print("Parameters transfer learning:", sum([param.nelement() for param in model.linear.parameters()]))

RobertaClassifier(
  (roberta): RobertaForSequenceClassification(
    (roberta): RobertaModel(
      (embeddings): RobertaEmbeddings(
        (word_embeddings): Embedding(50265, 768, padding_idx=1)
        (position_embeddings): Embedding(514, 768, padding_idx=1)
        (token_type_embeddings): Embedding(1, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): RobertaEncoder(
        (layer): ModuleList(
          (0-11): 12 x RobertaLayer(
            (attention): RobertaAttention(
              (self): RobertaSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): RobertaSelfOutput(
                (de

In [49]:
for epoch_num in range(2):
    total_acc_train = 0
    total_loss_train = 0

    model.train()
    for train_input, train_label in tqdm(train_loader):
        mask = train_input['attention_mask'].to(device)
        input_id = train_input['input_ids'].squeeze(1).to(device)
        train_label = train_label.to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, train_label)
        total_loss_train += batch_loss.item()

        acc = (output.argmax(dim=1) == train_label).sum().item()
        total_acc_train += acc

        model.zero_grad()
        batch_loss.backward()
        optimizer.step()

    model.eval()
    total_loss_val, total_acc_val = 0.0, 0.0
    for val_input, val_label in valid_loader:
        val_label = val_label.to(device)
        mask = val_input['attention_mask'].to(device)
        input_id = val_input['input_ids'].squeeze(1).to(device)

        output = model(input_id, mask)

        batch_loss = criterion(output, val_label)
        total_loss_val += batch_loss.item()

        acc = (output.argmax(dim=1) == val_label).sum().item()
        total_acc_val += acc

    print(
        f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_dataset): .3f} \
        | Train Accuracy: {total_acc_train / len(train_dataset): .3f} \
        | Val Loss: {total_loss_val / len(valid_dataset): .3f} \
        | Val Accuracy: {total_acc_val / len(valid_dataset): .3f}')

100%|██████████| 638/638 [01:10<00:00,  9.11it/s]


Epochs: 1 | Train Loss:  0.011         | Train Accuracy:  0.519         | Val Loss:  0.010         | Val Accuracy:  0.512


100%|██████████| 638/638 [01:09<00:00,  9.20it/s]


Epochs: 2 | Train Loss:  0.011         | Train Accuracy:  0.530         | Val Loss:  0.010         | Val Accuracy:  0.512
